<a href="https://colab.research.google.com/github/mohanrajmit/Transfer-Learning/blob/master/Transfer_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from pathlib import Path
import numpy as np
import joblib

# Empty lists to hold the images and labels for each each image
x_train = []
y_train = []

In [2]:
!git clone https://github.com/mohanrajmit/Transfer-Learning.git

Cloning into 'Transfer-Learning'...
remote: Enumerating objects: 2238, done.
remote: Counting objects: 100% (15/15), done.
remote: Compressing objects: 100% (15/15), done.
remote: Total 2238 (delta 8), reused 0 (delta 0), pack-reused 2223
Receiving objects: 100% (2238/2238), 13.87 MiB | 12.20 MiB/s, done.
Resolving deltas: 100% (14/14), done.


In [3]:
!ls

sample_data  Transfer-Learning


In [4]:
# Load the training data set by looping over every image file
for image_file in Path("Transfer-Learning/training_dataset").glob("**/*.png"):

    # Load the current image file
    image_data = load_img(image_file, target_size=(224, 224))

    # Convert the loaded image file to a numpy array
    image_array = img_to_array(image_data)

    # Add the current image to our list of training images
    x_train.append(image_array)

    # Add a label for this image. If it was a not_bird image, label it 0. If it was a bird, label it 1.
    if "not_bird" in image_file.stem:
        y_train.append(0)
    else:
        y_train.append(1)

In [5]:
print(np.array(x_train).shape)

(2000, 224, 224, 3)


In [6]:
# Convert the list of separate images into a single 4D numpy array. This is what Keras expects.
x_train = np.array(x_train)

# Normalize image data to 0-to-1 range
x_train = imagenet_utils.preprocess_input(x_train)



In [7]:

# Load the pre-trained neural network to use as a feature extractor
feature_extractor = VGG16(weights='imagenet', include_top=False,input_shape=(224,224,3))



58889256/58889256 [==============================] - 0s 0us/step


In [8]:
feature_extractor.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [ ]:
#feature_extractor_resnet = ResNet50(weights='imagenet', include_top=False,input_shape=(224,224,3))

94765736/94765736 [==============================] - 0s 0us/step


In [ ]:
feature_extractor.summary()

Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [9]:
#feature_extractor_resnet.summary()

In [10]:
# Extract features for each image (all in one pass)
features_x = feature_extractor.predict(x_train)



63/63 [==============================] - 19s 148ms/step


In [11]:
features_x.shape

(2000, 7, 7, 512)

In [12]:
#features_resnet = feature_extractor_resnet.predict(x_train)

In [13]:
#features_resnet.shape

In [14]:
Y_train = np.array(y_train)
print(x_train.shape[1:])
print(Y_train.shape)

(224, 224, 3)
(2000,)


In [15]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten

In [17]:
# Create a model_vgg and add layers
model_vgg = Sequential()

# Add layers to our model_vgg
model_vgg.add(Flatten(input_shape=features_x.shape[1:]))
model_vgg.add(Dense(128, activation='relu'))
model_vgg.add(Dropout(0.5))
model_vgg.add(Dense(1, activation='sigmoid'))

# Compile the model_vgg
model_vgg.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])


In [18]:
model_vgg.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_1 (Flatten)         (None, 25088)             0         
                                                                 
 dense_2 (Dense)             (None, 128)               3211392   
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense_3 (Dense)             (None, 1)                 129       
                                                                 
Total params: 3211521 (12.25 MB)
Trainable params: 3211521 (12.25 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [ ]:
# Create a model_vgg and add layers
'''model_resnet = Sequential()

# Add layers to our model_vgg
model_resnet.add(Flatten(input_shape=features_resnet.shape[1:]))
model_resnet.add(Dense(128, activation='relu'))
model_resnet.add(Dropout(0.5))
model_resnet.add(Dense(1, activation='sigmoid'))

# Compile the model_vgg
model_resnet.compile(loss='binary_crossentropy',optimizer='adam', metrics=['accuracy'])


In [33]:
# Train the model_vgg
model_vgg.fit(features_x,Y_train,epochs=200, verbose=2)

# Save the trained model_vgg to a file so we can use it to make predictions later
model_vgg.save("bird_feature_classifier_model_vgg.h5")

ValueError: ignored

In [27]:
# Train the model_vgg
'''model_resnet.fit(features_resnet,Y_train,epochs=10,shuffle=True, verbose=2)

# Save the trained model_vgg to a file so we can use it to make predictions later
model_resnet.save("bird_feature_classifier_model_resnet.h5")'''

'model_resnet.fit(features_resnet,Y_train,epochs=10,shuffle=True, verbose=2)\n\n# Save the trained model_vgg to a file so we can use it to make predictions later\nmodel_resnet.save("bird_feature_classifier_model_resnet.h5")'

In [28]:
from keras.models import load_model
from sklearn.metrics import confusion_matrix, classification_report

In [29]:
# Empty lists to hold the images and labels for each each image
x_test = []
y_test = []

# Load the test data set by looping over every image file
for image_file in Path("Transfer-Learning/test_dataset").glob("**/*.png"):

    # Load the current image file
    image_data = load_img(image_file, target_size=(224,224,3))

    # Convert the loaded image file to a numpy array
    image_array = img_to_array(image_data)

    # Add the current image to our list of test images
    x_test.append(image_array)

    # Add an expected label for this image. If it was a not_bird image, label it 0. If it was a bird, label it 1.
    if "not_bird" in image_file.stem:
        y_test.append(0)
    else:
        y_test.append(1)

# Convert the list of test images to a numpy array
x_test = np.array(x_test)

# Normalize test data set to 0-to-1 range
x_test = imagenet_utils.preprocess_input(x_test)

In [30]:
x_test.shape

(200, 224, 224, 3)

In [31]:
# Load our trained classifier model_vgg
model_vgg = load_model("bird_feature_classifier_model_vgg.h5")

# Extract features for each image (all in one pass)
features_x = feature_extractor.predict(x_test)

# Given the extracted features, make a final prediction using our own model_vgg
predictions = model_vgg.predict(features_x)

# If the model_vgg is more than 50% sure the object is a bird, call it a bird.
# Otherwise, call it "not a bird".
predictions = predictions > 0.5

# Calculate how many mis-classifications the model_vgg makes
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
print(f"True Positives: {tp}")
print(f"True Negatives: {tn}")
print(f"False Positives: {fp}")
print(f"False Negatives: {fn}")

# Calculate Precision and Recall for each class
report = classification_report(y_test, predictions)
print(report)

7/7 [==============================] - 0s 3ms/step
True Positives: 91
True Negatives: 84
False Positives: 16
False Negatives: 9
              precision    recall  f1-score   support

           0       0.90      0.84      0.87       100
           1       0.85      0.91      0.88       100

    accuracy                           0.88       200
   macro avg       0.88      0.88      0.87       200
weighted avg       0.88      0.88      0.87       200



In [ ]:
# Load our trained classifier model_vgg
'''model_resnet = load_model("bird_feature_classifier_model_resnet.h5")

# Extract features for each image (all in one pass)
features_resnet = feature_extractor_resnet.predict(x_test)

# Given the extracted features, make a final prediction using our own model_vgg
predictions = model_resnet.predict(features_resnet)

# If the model_vgg is more than 50% sure the object is a bird, call it a bird.
# Otherwise, call it "not a bird".
predictions = predictions > 0.5

# Calculate how many mis-classifications the model_vgg makes
tn, fp, fn, tp = confusion_matrix(y_test, predictions).ravel()
print(f"True Positives: {tp}")
print(f"True Negatives: {tn}")
print(f"False Positives: {fp}")
print(f"False Negatives: {fn}")

# Calculate Precision and Recall for each class
report = classification_report(y_test, predictions)
print(report)'''

7/7 [==============================] - 0s 5ms/step
True Positives: 88
True Negatives: 84
False Positives: 16
False Negatives: 12
              precision    recall  f1-score   support

           0       0.88      0.84      0.86       100
           1       0.85      0.88      0.86       100

    accuracy                           0.86       200
   macro avg       0.86      0.86      0.86       200
weighted avg       0.86      0.86      0.86       200

